# Coach Change Prediction

## 1. Project Context
As outlined in the project overview, the goal of this notebook is to address **Core Objective: Coach Changes**. We aim to predict the set of teams that will change coaches at the end of the test season (Year 10).

## 2. Methodology
To achieve this, we will model the problem as a **Binary Classification** task:
* **Target ($y$):** `1` if a team changes its coach between the current season and the next; `0` otherwise.
* **Features ($X$):** We will utilize historical basketball data including team performance metrics, coaching history, and player aggregates.

We will follow this workflow:
1.  **Data Loading:** Ingest relational tables (`teams`, `coaches`, `players`, etc.).
2.  **Target Engineering:** Construct the labeled target variable by looking ahead one season.
3.  **Feature Engineering:** Aggregate player awards, coach tenure, and derive advanced metrics (Win %, Point Differential).
4.  **Modeling:** Train and evaluate multiple classifiers (Random Forest, Gradient Boosting, SVM, etc.).
5.  **Prediction:** Generate predictions for the final test season (Year 10).

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

PYDEVD_DISABLE_FILE_VALIDATION=1

### 2.1 Load Data Sources
We utilize the project's relational database consisting of teams, coaches, and player statistics.
* **`teams`:** Team performance per season.
* **`coaches`:** Records of coaches managing teams.
* **`teams_post`:** Post-season results.
* **`players_teams`:** Player performance stats.
* **`awards_players`:** Awards received by players.

In [ ]:
teams_df = pd.read_csv("project_data/initial_data/teams.csv")
coaches_df = pd.read_csv("project_data/initial_data/coaches.csv")
teams_post_df = pd.read_csv("project_data/initial_data/teams_post.csv")
players_teams_df = pd.read_csv("project_data/initial_data/players_teams.csv")
awards_players_df = pd.read_csv("project_data/initial_data/awards_players.csv")

print(f"Loaded {len(teams_df)} team-season records.")
print(f"Loaded {len(coaches_df)} coach records.")
print(f"Loaded {len(teams_post_df)} post-season team records.")
print(f"Loaded {len(players_teams_df)} player records.")
print(f"Loaded {len(awards_players_df)} player award records.")

### 2.2 Target Variable Definition: `CoachChange`
We need to define a target variable that indicates if a coach will be replaced in the *following* season.

**Logic:**
1.  We group data by Team (`tmID`).
2.  We look at the `coachID` for the *next* year ($t+1$).
3.  If `coachID` at $t$ $\neq$ `coachID` at $t+1$, then `CoachChange = 1`.
4.  If the team folds (no record next year), we mark it for removal to avoid training noise.
5.  **Test Set:** The final year (Year 10) will have `NaN` as the target, as these are the values we must predict.

In [ ]:
coaches_labeled_df = coaches_df.copy()

# Keep only the last coach of each year
coaches_labeled_df = coaches_labeled_df.sort_values(by=['tmID', 'year'])

# Get the coachID from the next season
coaches_labeled_df['NextYearCoachID'] = coaches_labeled_df.groupby('tmID')['coachID'].shift(-1)

max_year = coaches_labeled_df['year'].max() 

conditions = [
    # Final Year
    (coaches_labeled_df['year'] == max_year),
    
    # Team folded
    (coaches_labeled_df['NextYearCoachID'].isnull()), 
    
    # Same Coach
    (coaches_labeled_df['coachID'] == coaches_labeled_df['NextYearCoachID']),
    
    # Different Coach
    (coaches_labeled_df['coachID'] != coaches_labeled_df['NextYearCoachID'])
]

choices = [
    np.nan, # Keep as NaN (Test Set)
    -1,     # Mark for Removal (Folded Team)
    0,      # No Change
    1       # Change
]

coaches_labeled_df['CoachChange'] = np.select(conditions, choices)
coaches_labeled_df = coaches_labeled_df[coaches_labeled_df['CoachChange'] != -1]

coach_labels_df = coaches_labeled_df[['tmID', 'year', 'CoachChange']]

print("\n--- Target Variable 'CoachChange' Created ---")
print(coach_labels_df[coach_labels_df['CoachChange'] == np.nan])
print(f"\nTotal 'CoachChange=1' events: {int(coach_labels_df['CoachChange'].sum())}")
print(f"Total 'CoachChange=np.nan' events: {int(coach_labels_df['CoachChange'].isnull().sum())}")
print(f"Total entries in max_year({max_year}): {(coach_labels_df['year'] == max_year).sum()}")

### 3. Feature Engineering & Data Assembly
To improve prediction accuracy, we need to move beyond raw stats. We will engineer features that reflect the "status" of the team and the coach:

1.  **Player Talent:** We quantify the talent level by counting the number of individual awards (`awards_players`) a team's players won in a given season.
2.  **Coach History:** We calculate `stint_max` (how long the coach has been there) and their historical post-season success (`coach_post_wins`).
3.  **Data Merge:** We aggregate these features into a single analytical table (`final_df`).

In [ ]:
# --- 3. Engineer and Merge Feature Sets ---
stats_single_team = (players_teams_df
                     .sort_values(['playerID', 'year', 'stint'])
                     .drop_duplicates(subset=['playerID', 'year'], keep='last')
                     [['playerID', 'year', 'tmID']]
)

awards_with_team_df = (
    awards_players_df
    .merge(stats_single_team, on=['playerID', 'year'])
)

# 3a. Create 'num_player_awards'
awards_count_df = (
    awards_with_team_df
    .groupby(['tmID', 'year'])
    .size()
    .reset_index(name='num_player_awards')
)

# 3b. A team might have multiple coaches (stints). We'll sum their post-season stats and take the max stint number.
coach_features_agg = coaches_df.groupby(['tmID', 'year']).agg(
    stint_max=('stint', 'max'),
    coach_post_wins=('post_wins', 'sum'),
    coach_post_losses=('post_losses', 'sum')
).reset_index()

# 3c. Get team-specific post-season features
teams_post_features = teams_post_df[['tmID', 'year', 'W', 'L']].rename(columns={
    'W': 'team_post_W',
    'L': 'team_post_L'
})

# 3d. Assemble the final DataFrame
# Start with the base team stats
final_df = teams_df.copy()

# Merge the target variable (CoachChange)
final_df = pd.merge(final_df, coach_labels_df, on=['tmID', 'year'], how='left')

# Merge the aggregated coach features
final_df = pd.merge(final_df, coach_features_agg, on=['tmID', 'year'], how='left')

# Merge the team post-season features
final_df = pd.merge(final_df, teams_post_features, on=['tmID', 'year'], how='left')

# Merge the player award count
final_df = pd.merge(final_df, awards_count_df, on=['tmID', 'year'], how='left')

# --- 4. Clean Up Merged Data ---
# Merges create 'NaN' for teams that didn't make playoffs or win awards.
# We'll fill these with '0' as 'NaN' means 'zero wins' or 'zero awards'.
fill_zero_cols = [
    'coach_post_wins', 'coach_post_losses', 'team_post_W',
    'team_post_L', 'num_player_awards'
]
for col in fill_zero_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0)

print("\n--- Final Assembled DataFrame ---")
print(final_df.head())
print(f"\nFinal DataFrame shape: {final_df.shape}")
print(f"Columns: {final_df.columns.to_list()}")

### 3.1 Derived Performance Metrics
We convert raw counts into standardized ratios to compare teams across different numbers of games played (`GP`):
* **Win Percentage:** $\frac{Wins}{Wins + Losses}$
* **Point Differential:** `o_pts` (Offensive Points) - `d_pts` (Defensive Points).
* **Playoff Flag:** A binary indicator of whether the team qualified for the post-season.

In [ ]:
# 1. Create Win Percentage
# We add a small number (1e-6) to avoid division by zero for any (unlikely) 0-game seasons
final_df['win_pct'] = final_df['won'] / (final_df['won'] + final_df['lost'] + 1e-6)
final_df['home_win_pct'] = final_df['homeW'] / (final_df['homeW'] + final_df['homeL'] + 1e-6)
final_df['conf_win_pct'] = final_df['confW'] / (final_df['confW'] + final_df['confL'] + 1e-6)

# 2. Create Point Differential
final_df['pt_diff'] = final_df['o_pts'] - final_df['d_pts']

# 3. Create a simple 'made_playoffs' flag
# If a team has any team_post_W or team_post_L, they were in the post-season.
final_df['made_playoffs'] = ((final_df['team_post_W'] > 0) | (final_df['team_post_L'] > 0)).astype(int)

print("--- DataFrame with Engineered Features (sample) ---")
print(final_df[['tmID', 'year', 'win_pct', 'pt_diff', 'made_playoffs', 'CoachChange']].head())

### 4. Data Preparation
Before modeling, we must clean the feature set to prevent **Data Leakage** and remove redundancy.

**Preprocessing Steps:**
1.  **Drop Leakage/IDs:** Remove identifiers (`tmID`, `name`) and columns that directly reveal the target.
2.  **Remove Redundancy:** Drop raw counters (like `won`, `lost`) in favor of the calculated percentages. Remove highly correlated stat components (e.g., keeping `fga` but dropping `fgm` as they track closely).
3.  **Split:** Separate the dataset into:
    * **`X` & `y`:** Historical data (Years 1-9) for training/validation.
    * **`X_test_predict`:** The final year (Year 10) for the submission.

In [ ]:
# 1. Separate the target 'y' and the 'test_df'
train_df = final_df.dropna(subset=['CoachChange'])
y = train_df['CoachChange']

test_df = final_df[final_df['CoachChange'].isnull()]
test_df = test_df.drop_duplicates(subset=['tmID', 'year'], keep='last')

# --- Team IDs for the Final Submission ---
teams_test_final = test_df[['tmID', 'year']].reset_index(drop=True)

# 2. Define features to drop
features_to_drop = [
    # --- Target/Leakage ---
    'CoachChange',

    # --- ID / Text Columns ---
    'tmID', 'lgID', 'franchID', 'confID', 'divID', 'name', 'arena',

    # --- Replaced by win_pct ---
    'won', 'lost', 'homeW', 'homeL', 'awayW', 'awayL', 'confW', 'confL',
    'conf_win_pct', # Correlated with win_pct

    # --- Replaced by pt_diff (which we ALSO drop) ---
    'o_pts', 'd_pts',
    'pt_diff', # Correlated with win_pct

    # --- 100% Redundant Sums (from teams table) ---
    'o_reb', 'd_reb', 'tmTRB', 'opptmTRB',
    'tmORB', 'tmDRB', 'opptmORB', 'opptmDRB', # Keep o_oreb, o_dreb, etc.

    # --- Redundant Components ---
    'o_fta', 'o_3pa', 'd_fta', 'd_3pa', # Keep 'o_fga' and 'd_fga' as primary
    'o_fgm', 'o_ftm', 'o_3pm', # Drop 'made' stats, keep 'attempted'
    'd_fgm', 'd_ftm', 'd_3pm',

    # --- Other Redundancies ---
    'min',               # Correlated with GP
    'made_playoffs',     # Redundant with 'playoff' column
    'team_post_W',       # Redundant with coach_post_wins
    'team_post_L',       # Redundant with coach_post_losses
    'seeded'             # Redundant with rank
]

# 3. Create the features DataFrames
X = train_df.drop(columns=features_to_drop, errors='ignore')
X_test_predict = test_df.drop(columns=features_to_drop, errors='ignore')

# Convert Y/N columns to 1/0
cols_to_map = ['playoff', 'firstRound', 'semis', 'finals']
for col in cols_to_map:
    if col in X.columns:
        X[col] = X[col].map({'Y': 1, 'N': 0})
        X_test_predict[col] = X_test_predict[col].map({'Y': 1, 'N': 0})

# 4. Handle Missing Values
X = X.fillna(0)
X_test_predict = X_test_predict.fillna(0)

# --- VARIABLE 2 TO SAVE: Aligned Feature Matrix for Prediction ---
# Ensure test columns match training columns exactly (prevents shape mismatch errors)
X_test_predict = X_test_predict.reindex(columns=X.columns, fill_value=0)

print(f"--- Final 'X' (Features) DataFrame ---")
print(f"Shape of X: {X.shape}")
print(f"Shape of X_test_predict: {X_test_predict.shape}")
print(f"Features: {X.columns.to_list()}\n")

print(f"--- Final 'y' (Target) Series ---")
print(f"Shape of y: {y.shape}")
print(f"Class distribution:\n{y.value_counts()}")

### 5. Exploratory Data Analysis (Correlation)
We check for multicollinearity among features. High correlation (e.g., > 0.9) between features can confuse linear models and inflate feature importance in tree models.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# --- C. Check Correlation ---

# Calculate the correlation matrix
corr_matrix = X.corr().abs()

# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=False, cmap='Blues', fmt='.1f')
plt.title('Feature Correlation Matrix')
plt.show()

# You can also manually find high-correlation pairs
# Select upper triangle of correlation matrix
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.9
high_corr_features = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

if high_corr_features:
    print(f"\nWARNING!: High Correlation remaining in features: {high_corr_features}")
    print("Consider dropping one feature from each correlated pair.")
else:
    print("\nNo highly correlated (r > 0.9) features found. Ready for modeling.")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight

def print_metrics(y_true, y_pred, model_name):
    """Prints common classification metrics."""
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    
    print(f"\n--- {model_name} Metrics ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-Score: {f1:.4f}")
    return {"Model": model_name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}

In [ ]:
test_year = 9
starting_year = 3
max_year = 9
random_state = 45

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
import itertools

# Validation Strategy: Leave-One-Season-Out
years = range(starting_year, max_year + 1)
metrics = []

for test_year in years:
    print(f"Year: {test_year}")
    # Split data
    X_train = X[X['year'] < test_year]
    y_train = y[X['year'] < test_year]
    X_test = X[X['year'] == test_year]
    y_test = y[X['year'] == test_year]

    X_train = X_train.drop(columns=['year'])
    X_test = X_test.drop(columns=['year'])

    # Scale data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Compute sample weights
    sample_weights = compute_sample_weight(class_weight='balanced', y=y_train)

    # --- Logistic Regression ---

    param_grid_lr = {
        'C': [0.01, 0.05, 0.1, 1, 10], 
        'class_weight': ['balanced']
    }

    lr = LogisticRegression(max_iter=1000, random_state=random_state)

    grid_lr = GridSearchCV(lr, param_grid_lr, cv=5, scoring='f1') 
    grid_lr.fit(X_train_scaled, y_train)
    best_lr = grid_lr.best_estimator_

    y_pred_lr = best_lr.predict(X_test_scaled)

    lr_metrics = print_metrics(y_test, y_pred_lr, 'Logistic Regression')
    print(f"Best LR Params: {grid_lr.best_params_}")


    # --- Random Forest ---

    param_grid_rf = {
        'max_depth': [3, 5], 
        'n_estimators': [100, 200, 300],
        'class_weight': ['balanced', 'balanced_subsample'],
        'min_samples_leaf': [2, 4, 5],
        'max_features': ['sqrt']
    }

    rf = RandomForestClassifier(random_state=42)
    grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='f1', n_jobs=-1)
    grid_rf.fit(X_train_scaled, y_train)
    best_rf = grid_rf.best_estimator_

    y_pred_rf = best_rf.predict(X_test_scaled)

    rf_metrics = print_metrics(y_test, y_pred_rf, 'Random Forest')
    print(f"Best RF Params: {grid_rf.best_params_}")
    

    # --- Gradient Boosting ---
    param_grid_gb = { 
        'n_estimators': [100, 200, 500, 1000], 
        'learning_rate': [0.0005, 0.001, 0.01, 0.05], 
        'max_depth': [2, 3, 5],
    }

    gb = GradientBoostingClassifier(random_state=42)

    grid_gb = GridSearchCV(gb, param_grid_gb, cv=5, scoring='f1', n_jobs=-1)
    grid_gb.fit(X_train_scaled, y_train, sample_weight=sample_weights)
    best_gb = grid_gb.best_estimator_

    best_gb.fit(X_train_scaled, y_train, sample_weight=sample_weights)

    y_pred_gb = best_gb.predict(X_test_scaled)

    gb_metrics = print_metrics(y_test, y_pred_gb, 'Gradient Boosting')
    print(f"Best GB Params: {grid_gb.best_params_}")


    # --- Support Vector Machine ---

    param_grid_svm = {
        'C': [0.01, 0.1, 1, 10, 50], 
        'gamma': ['scale', 'auto'], 
        'kernel': ['sigmoid'],
        'class_weight': ['balanced']
    }

    svm = SVC(probability=True, random_state=42)
    grid_svm = GridSearchCV(svm, param_grid_svm, cv=5, scoring='f1', n_jobs=-1)

    grid_svm.fit(X_train_scaled, y_train)
    best_svm = grid_svm.best_estimator_

    y_pred_svm = best_svm.predict(X_test_scaled)

    svm_metrics = print_metrics(y_test, y_pred_svm, 'SVM')
    print(f"Best SVM Params: {grid_svm.best_params_}")


    # --- Decision Tree (Single, Tuned) ---

    param_grid_dt = {
        'max_depth': [3, 5, 7], 
        'min_samples_leaf': [1, 2, 4], 
        'class_weight': ['balanced']
    }

    dt = DecisionTreeClassifier(random_state=42)
    grid_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='f1', n_jobs=-1)
    grid_dt.fit(X_train_scaled, y_train)
    best_dt = grid_dt.best_estimator_

    y_pred_dt = best_dt.predict(X_test_scaled)

    dt_metrics = print_metrics(y_test, y_pred_dt, 'Decision Tree')
    print(f"Best DT Params: {grid_dt.best_params_}")

    voting_clf = VotingClassifier(
        estimators=[
            ('lr', best_lr), 
            ('rf', best_rf), 
            ('gb', best_gb), 
            ('svm', best_svm),
        ],
    )

    weights_list = [
        [1, 1, 1, 1], # Baseline (Equal)
        [1, 2, 2, 1], # Favor Trees (Precision)
        [2, 1, 1, 2], # Favor SVM & LR (Recall)
        [0, 2, 2, 1], # Drop LR, favor Trees
        [0, 1, 1, 1], # Drop LR, Equal others
        [1, 3, 3, 1], # Aggressively favor Trees
        [1, 2, 1, 2]  # RF + SVM partnership
    ]

    param_grid_voting = {
        'weights': weights_list,
        'voting': ['soft']
    }

    grid_voting = GridSearchCV(
        estimator=voting_clf,
        param_grid=param_grid_voting,
        cv=10,
        scoring='recall',
        n_jobs=-1,
        verbose=1
    )

    print("Tuning Voting Classifier Weights...")
    grid_voting.fit(X_train_scaled, y_train)

    best_voting_clf = grid_voting.best_estimator_
    print(f"\nBest Weights: {grid_voting.best_params_}")

    y_pred_voting = best_voting_clf.predict(X_test_scaled)

    # clf_metrics = print_metrics(y_test, y_pred_voting, 'Tuned Voting Classifier')

    all_predictions = {'Logistic Regression': y_pred_lr, 'Random Forest': y_pred_rf, 'Gradient Boosting': y_pred_gb, 'SVM': y_pred_svm, 'Decision Tree': y_pred_dt, 'Voting Classifier': y_pred_voting}
    all_metrics = [lr_metrics, rf_metrics, gb_metrics, svm_metrics, dt_metrics]
    final_results_df = pd.DataFrame(all_metrics).set_index('Model').sort_values(by='F1', ascending=False)

    print(f"\n--- Final Metrics Comparison (Test Year {test_year}) ---")
    print(final_results_df)

    right_predictions = {}
    wrong_predictions = {}

    for name, y_pred in all_predictions.items():
        # Predict & Rank
        test_data = train_df[train_df['year'] == test_year].copy()
        test_data['PredictedCoachChange'] = y_pred
        
        results = test_data[['tmID', 'PredictedCoachChange', 'CoachChange']].copy()
        
        TP_lines = results[(results['PredictedCoachChange'] == 1) & (results['CoachChange'] == 1)]
        TN_lines = results[(results['PredictedCoachChange'] == 0) & (results['CoachChange'] == 0)]

        right_predictions[test_year] = {
            'True Positives': TP_lines,
            'True Negatives': TN_lines
        }

        FP_lines = results[(results['PredictedCoachChange'] == 1) & (results['CoachChange'] == 0)]
        FN_lines = results[(results['PredictedCoachChange'] == 0) & (results['CoachChange'] == 1)]

        wrong_predictions[test_year] = {
            'False Positives': FP_lines,
            'False Negatives': FN_lines
        }

        print(f"--- Year {test_year} Summary ---")

        print("\n## ✅ RIGHT PREDICTIONS")
        print("### True Positives (Predicted Change, Actual Change):")
        display(TP_lines)
        
        print("### True Negatives (Predicted NO Change, Actual NO Change):")
        display(TN_lines)
        
        print("\n" + "-"*30)
        
        ## ❌ Model Got It WRONG
        print("## ❌ WRONG PREDICTIONS")
        print("### False Positives (Predicted Change, Actual NO Change - Type I Error):")
        display(FP_lines)
        
        print("### False Negatives (Predicted NO Change, Actual Change - Type II Error/Missed):")
        display(FN_lines)

        metrics.append({
            'Year': test_year,
            'Model': name,
            'TP': len(TP_lines),
            'TN': len(TN_lines),
            'FP': len(FP_lines),
            'FN': len(FN_lines),
            'Precision': TP_lines.shape[0] / (TP_lines.shape[0] + FP_lines.shape[0]),
            'Recall': TP_lines.shape[0] / (TP_lines.shape[0] + FN_lines.shape[0]),
            'F1': 2 * (TP_lines.shape[0] / (TP_lines.shape[0] + FP_lines.shape[0])) * (TP_lines.shape[0] / (TP_lines.shape[0] + FN_lines.shape[0])) / ((TP_lines.shape[0] / (TP_lines.shape[0] + FP_lines.shape[0])) + (TP_lines.shape[0] / (TP_lines.shape[0] + FN_lines.shape[0])))
        })

metrics_df = pd.DataFrame(metrics)
metrics_df.sort_values(by='F1', ascending=False)
display(metrics_df)

In [ ]:
metrics_df = metrics_df.sort_values(by='F1', ascending=False)

display(metrics_df[metrics_df['Year'] == 7].style.background_gradient(cmap='Greens', subset=['F1', 'Precision', 'Recall']))
display(metrics_df[metrics_df['Year'] == 8].style.background_gradient(cmap='Greens', subset=['F1', 'Precision', 'Recall']))
display(metrics_df[metrics_df['Year'] == 9].style.background_gradient(cmap='Greens', subset=['F1', 'Precision', 'Recall']))